<p>
  <a href="https://colab.research.google.com/github/neo4j-partners/hands-on-lab-neo4j-and-vertex-ai/blob/main/Lab%205%20-%20Graph%20Data%20Science/embedding.ipynb" target="_blank">
    <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
  </a>
</p>

# Install Prerequisites
First off, you'll also need to install a few packages.

In [ ]:
!pip install --quiet --upgrade neo4j
!pip install --quiet google-cloud-storage

# Working with Neo4j
You'll need to enter the credentials from your Neo4j instance below.

The default DB_NAME is always neo4j.

In [ ]:
DB_URL = 'neo4j://35.237.130.165:7687'
DB_USER = 'neo4j'
DB_PASS = 'foo123'
DB_NAME = 'neo4j'

In [ ]:
import pandas as pd
from neo4j import GraphDatabase

driver = GraphDatabase.driver(DB_URL, auth=(DB_USER, DB_PASS))

First we're going to create an in memory graph represtation of the data in Neo4j Graph Data Science (GDS).

In [ ]:
with driver.session(database=DB_NAME) as session:
  result = session.read_transaction(
    lambda tx: tx.run(
      """
CALL gds.graph.project(
    'mygraph',
    ['Company', 'Manager', 'Holding'],
    {
        OWNS: {orientation: 'UNDIRECTED'},
        PARTOF: {orientation: 'UNDIRECTED'}
    }
)
YIELD
    graphName AS graph,
    relationshipProjection AS readProjection,
    nodeCount AS nodes,
    relationshipCount AS rels
      """
    ).data()
  )
df = pd.DataFrame(result)
display(df)

Note, if you get an error saying the graph already exists, that's probably because you ran this code before. You can destroy it using this command:

In [ ]:
with driver.session(database=DB_NAME) as session:
  result = session.read_transaction(
    lambda tx: tx.run(
      """
      CALL gds.graph.drop('mygraph')
      """
    ).data()
  )

Now, let's list the details of the graph to make sure the projection was created as we want.

In [ ]:
with driver.session(database=DB_NAME) as session:
  result = session.read_transaction(
    lambda tx: tx.run(
      """
      CALL gds.graph.list()
      """
    ).data()
  )
  print(result)

Now we can generate an embedding from that graph. This is a new feature we can use in our predictions. We're using FastRP, which is a more full featured and higher performance of Node2Vec. You can learn more about that [here](https://neo4j.com/docs/graph-data-science/current/algorithms/fastrp/).

In [ ]:
with driver.session(database=DB_NAME) as session:
  result = session.read_transaction(
    lambda tx: tx.run(
      """
        CALL gds.fastRP.mutate('mygraph',{
        embeddingDimension: 16,
        randomSeed: 1,
        mutateProperty:'embedding'
        })
      """
    ).data()
  )
df = pd.DataFrame(result)
display(df)

That creates an embedding for each node type.  However, we only want the embedding on the nodes of type holding.

We're going to take the embedding from our projection and write it to the holding nodes in the underlying database.

In [ ]:
with driver.session(database=DB_NAME) as session:
  result = session.run(
    """
      CALL gds.graph.writeNodeProperties('mygraph', ['embedding'], ['Holding'])
      YIELD writeMillis
    """
  )
  print(result)

In [ ]:
with driver.session(database=DB_NAME) as session:
  result = session.read_transaction(
    lambda tx: tx.run(
      """
        MATCH (n:Holding) RETURN n
      """
    ).data()
  )

Note that this query will take 2-3 minutes to run as it's grabbing nearly half a million nodes along with all their properties and our new embedding.

In [ ]:
df = pd.DataFrame([dict(record.get('n')) for record in result])
df

Note that the embedding row is an array. To make this dataset more consumable, we should flatten that out into multiple individual features: embedding_0, embedding_1, ... embedding_n.


In [ ]:
embeddings = pd.DataFrame(df['embedding'].values.tolist()).add_prefix("embedding_")
merged = df.drop(columns=['embedding']).merge(embeddings, left_index=True, right_index=True)
merged

Now that we have the data formatted properly, let's split it into a training and a testing set and write those to disk.

In [ ]:
df = merged

train = df.loc[df['reportCalendarOrQuarter'] == '03-31-2021']
train = train.append(df.loc[df['reportCalendarOrQuarter'] == '06-30-2021'])
train.to_csv('train.csv', index=False)

test = df.loc[df['reportCalendarOrQuarter'] == '09-30-2021']
test.to_csv('test.csv', index=False)

# Authenticate your Google Cloud Account
Now let's write the file to Google Cloud Storage so we can use it in our model.  To do so, we must first authenticate.

In [ ]:
# Edit these variables!
PROJECT_ID = 'YOUR-PROJECT-ID'
STORAGE_BUCKET = 'NAME-OF-BUCKET-TO-CREATE'

# You can leave these defaults
REGION = 'us-central1'

In [ ]:
import os
os.environ['GCLOUD_PROJECT'] = PROJECT_ID

In [ ]:
try:
    from google.colab import auth as google_auth
    google_auth.authenticate_user()
except:
    pass

# Upload to Google Cloud Storage
Now we can upload our data sets to our bucket.

In [ ]:
from google.cloud import storage
client = storage.Client()

In [ ]:
bucket = client.bucket(STORAGE_BUCKET)
bucket.location=REGION
client.create_bucket(bucket)

In [ ]:
# Upload our files to that bucket
for filename in ['train.csv', 'test.csv']:
    upload_path = os.path.join('embedding', filename)
    blob = bucket.blob(upload_path)
    blob.upload_from_filename(filename)